# Rpy2 2x2x3 Mixed ANOVA Demo

Note: this works with a mix of 1 or more within and/or 1 or more between subjects factors.


### Get Ready

- First you have to have R installed and working on your system.
- Then you have to pip install rpy2, pandas and other packages needed for python data analysis (see stats_requirements.txt)
- Then try to run this stuff...

### ENVIRONMENT SETUP

In [1]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import seaborn as sns
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt
import logging
from IPython.display import Image, HTML

from rpy2.robjects import pandas2ri
import rpy2.robjects as ro

# %matplotlib inline
%load_ext rpy2.ipython
pandas2ri.activate() # allows you to move dataframes in/out of pandas using ro.r()


In [2]:
# IMPORT CONFIGS

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')

## R Package Setup

### Only need to run this once per R-install (this can take a long time to run)

If this doesn't work here, try running them in R directly and then reload this notebook

In [3]:
run_this = False

if run_this:
    ro.r(
        f"""
        install.packages('lme4')
        install.packages('car')
        install.packages('sjstats')
        install.packages('emmeans')
        install.packages('effects')
        """
    )

In [4]:
# note: I use a personal library as I'm not able to install packages to the default /usr/local/lib/R/site-library
#       If you don't have this problem, then just use the simpler version above!
import getpass

run_this = False

personal_library = f'/home/{getpass.getuser()}/R/x86_64-pc-linux-gnu-library/4.4'

if run_this:
    ro.r(
        f"""
        install.packages('lme4', lib="{personal_library}")
        install.packages('car', lib="{personal_library}")
        install.packages('sjstats', lib="{personal_library}")
        install.packages('emmeans', lib="{personal_library}")
        install.packages('effects', lib="{personal_library}")
        """
    )

### set some R options

In [5]:
%%R 
options(scipen = 50, digits = 5)
options("width"=200)
options("warn=-1") # issue "warn=0" to turn warnings back on

$`warn=-1`
NULL



/home/nogard/GLOBAL_VENVS/stats/lib/python3.10/site-packages/rpy2/ipython/rmagic.py:990: DeprecationWarning: The `source` parameter emit a  deprecation warning since IPython 8.0, it had no effects for a long time and will  be removed in future versions.
  displaypub.publish_display_data(


In addition: Warning message:
In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
  library ‘/usr/lib/R/site-library’ contains no packages


## Create Some Sample Data

In [6]:
np.random.seed(1138)  # For reproducibility

# create fake data
# ----------------

subject_datas = []
for i in range(40):
    df = pd.DataFrame({
        'Subject': f'Subject{i + 1}',
        'Condition': np.repeat(['Memory', 'NoMemory'], 12),
        'Difficulty': np.tile(np.repeat(['Easy', 'Hard'], 6), 2),
        'Session': np.tile(np.tile([1, 2, 3], 4), 2),
        'RT': np.random.normal(0, 1, 24) * 100
    })
    subject_datas.append(df)
data = pd.concat(subject_datas)
data.reset_index(inplace=True)
# force significant effects
# -------------------------

indices = (data['Condition'] == 'Memory') & (data['Difficulty'] == 'Hard')
data.loc[indices, 'RT'] += 28.78  # Significant effect

indices = (data['Session'] == 2) & (data['Difficulty'] == 'Easy')
data.loc[indices, 'RT'] += 35

indices = (data['Session'] == 3) & (data['Difficulty'] == 'Easy')
data.loc[indices, 'RT'] += 46

# show
# ------

display(data.head())
print()
display(data.tail())


,index,Subject,Condition,Difficulty,Session,RT
0,0,Subject1,Memory,Easy,1,20.412906
1,1,Subject1,Memory,Easy,2,59.068017
2,2,Subject1,Memory,Easy,3,117.022896
3,3,Subject1,Memory,Easy,1,-185.324828
4,4,Subject1,Memory,Easy,2,-22.216210


,index,Subject,Condition,Difficulty,Session,RT
955,19,Subject40,NoMemory,Hard,2,-138.989539
956,20,Subject40,NoMemory,Hard,3,-62.120561
957,21,Subject40,NoMemory,Hard,1,-130.726300
958,22,Subject40,NoMemory,Hard,2,-5.398787
959,23,Subject40,NoMemory,Hard,3,18.513640


## Analysis using lme4 for the anova

This approach involves just running normal R code directly. The only other thing to do is first put your data into the global R space, then run your R analysis code as you would in R. One downside is that the error messages you can can often be unhelpful or misleading. Otherwise, this is the way I run R analyses from python.

In [7]:
import rpy2.robjects as ro

ro.globalenv['data'] = data
ro.r(
    """
    require(lme4)
    require(car)
    require(sjstats)
    require(emmeans)
    require(effects)

    ## Convert Factor variables to factors
    
    data$Subject <- factor(data$Subject)
    data$Condition <- factor(data$Condition)
    data$Difficulty <- factor(data$Difficulty)
    data$Session <- factor(data$Session)

    ## OPTIONALLY Set orthogonal contrasts.
    op <- options(contrasts = c("contr.helmert", "contr.poly"))

    print("Mixed Effects ANOVA")
    model <- lmer(RT ~ Condition * Difficulty * Session + (1|Subject), data=data)
    print(anova(model))
    print("vvvvv This table shows p-values with type 3 sum of squares")
    anova_result <- Anova(model, type=3)  # type III sum of squares
    print(anova_result)
    
    print("")
    print("Marginal Means")
    ## Get marginal means
    means <- emmeans(model, ~ Condition * Difficulty * Session)
    print(means)

    # print("")
    # print("Pairwise Comparisons -- TUKEY")
    # comparisons <- pairs(means, adjust="tukey", infer=c("conf.int"))
    # print(comparisons)
    
    print("")
    print("Pairwise Comparisons -- HOLM")
    comparisons <- pairs(means, adjust="holm", infer=c("conf.int"))
    print(comparisons)
    
    # NOTE: ^^^ other pairs adjust options are "sidak", "bonferroni", "none", "mvt", "scheffe" and "dunnett"
    
    print("")
    print("confidence intervals")
    print(confint(model))
        
    # This plots marginal effects and interaction effects (disabled here)
    # plot(allEffects(model))
    
    ## Re-Set options to previous values
    options(op)
    """
)

R[write to console]: Loading required package: lme4

R[write to console]: Loading required package: Matrix

R[write to console]: Loading required package: car

R[write to console]: Loading required package: carData

R[write to console]: Loading required package: sjstats

R[write to console]: Loading required package: emmeans

R[write to console]: Loading required package: effects

R[write to console]: lattice theme set by effectsTheme()
See ?effectsTheme for details.



[1] "Mixed Effects ANOVA"


R[write to console]: boundary (singular) fit: see help('isSingular')



Analysis of Variance Table
                             npar Sum Sq Mean Sq F value
Condition                       1  28349   28349    2.88
Difficulty                      1   5869    5869    0.60
Session                         2 122543   61271    6.22
Condition:Difficulty            1  17525   17525    1.78
Condition:Session               2  11661    5830    0.59
Difficulty:Session              2 100151   50076    5.09
Condition:Difficulty:Session    2   7929    3964    0.40
[1] "vvvvv This table shows p-values with type 3 sum of squares"
Analysis of Deviance Table (Type III Wald chisquare tests)

Response: RT
                             Chisq Df        Pr(>Chisq)    
(Intercept)                  57.84  1 0.000000000000028 ***
Condition                     2.88  1            0.0897 .  
Difficulty                    0.60  1            0.4400    
Session                      12.45  2            0.0020 ** 
Condition:Difficulty          1.78  1            0.1821    
Condition:Session  

R[write to console]: Computing profile confidence intervals ...



                                    2.5 %    97.5 %
.sig01                            0.00000  11.25280
.sigma                           94.33821 103.16874
(Intercept)                      18.10997  30.59545
Condition1                      -11.67692   0.80856
Difficulty1                      -8.71530   3.77018
Session1                          3.49354  18.78507
Session2                          0.32522   9.15379
Condition1:Difficulty1          -10.51540   1.97008
Condition1:Session1              -5.39071   9.90082
Condition1:Session2              -6.50669   2.32188
Difficulty1:Session1            -16.88743  -1.59590
Difficulty1:Session2             -9.28179  -0.45322
Condition1:Difficulty1:Session1  -8.97152   6.32001
Condition1:Difficulty1:Session2  -2.53183   6.29673


R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In nextpar(mat, cc, i, delta, lowcut, upcut) :
R[write to console]: 
 
R[write to console]:  Last two rows have identical or NA .zeta values: using minstep

R[write to console]: 2: 
R[write to console]: In nextpar(mat, cc, i, delta, lowcut, upcut) :
R[write to console]: 
 
R[write to console]:  Last two rows have identical or NA .zeta values: using minstep

R[write to console]: 3: 
R[write to console]: In FUN(X[[i]], ...) :
R[write to console]:  non-monotonic profile for .sig01

R[write to console]: 4: 
R[write to console]: In confint.thpr(pp, level = level, zeta = zeta) :
R[write to console]: 
 
R[write to console]:  bad spline fit for .sig01: falling back to linear interpolation

R[write to console]: 5: 
R[write to console]: In regularize.values(x, y, ties, missing(ties), na.rm = na.rm) :
R[write to console]: 
 
R[write to console]:  collapsing to unique 'x' values

### Another approach that uses `aov` instead of `lme4` to run the anova. 

One upside is that the output table has p-values built in, however it is not as easy to get post-hocs with this version (well, maybe it is, but I don't know how to do it at the moment).

In [8]:

ro.globalenv['data'] = data
ro.r(
    """
    require(lme4)
    require(car)
    require(sjstats)
    require(graphics)
    
    ## Convert Factor variables to factors
    
    data$Subject <- factor(data$Subject)
    data$Condition <- factor(data$Condition)
    data$Difficulty <- factor(data$Difficulty)
    data$Session <- factor(data$Session)
    
    ## Set OPTIONAL orthogonal contrasts.
    op <- options(contrasts = c("contr.helmert", "contr.poly"))
    
    model = aov(RT ~ Condition * Difficulty * Session + Error(Subject), data=data)
    print(summary(model, type=3))
    print(coefficients(model))
        
    ## Re-Set options to previous values
    options(op)
    """
    )
    


Error: Subject
          Df Sum Sq Mean Sq F value Pr(>F)
Residuals 39 287403    7369               

Error: Within
                              Df  Sum Sq Mean Sq F value Pr(>F)   
Condition                      1   28349   28349    2.85 0.0917 . 
Difficulty                     1    5869    5869    0.59 0.4427   
Session                        2  122543   61271    6.16 0.0022 **
Condition:Difficulty           1   17525   17525    1.76 0.1848   
Condition:Session              2   11661    5830    0.59 0.5567   
Difficulty:Session             2  100151   50076    5.03 0.0067 **
Condition:Difficulty:Session   2    7929    3964    0.40 0.6715   
Residuals                    909 9043586    9949                  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
(Intercept) :
(Intercept) 
     24.353 

Subject :
numeric(0)

Within :
                     Condition1                     Difficulty1                        Session1                        Session2          Cond

## NOTE: This Pythonic interface for Rpy2

It works, but I find there are lots of quirks (see warnings and error at end converting confidence intervals to a table).

More importantly, in addition to R, I now have to learn Rpy2's appraoch, including, e.g., knowing when to use Formula(), knowing that base objects like the `anova` command need to be reference using `ro.r()`, etc. It's better to just use `ro.r(''' my R code here ''')` and include regular R code in the string.

In [9]:
# Convert pandas DataFrame to R DataFrame

# THESE ARE ALREADY DEFINED AT THE TOP, HERE JUST IN CASE YOU START ON THIS CELL?
# import rpy2.robjects as ro
# from rpy2.robjects import pandas2ri
# pandas2ri.activate() # allows you to move dataframes in/out of pandas using ro.r()

# THESE ARE ONLY NEEEDED FOR THE CODE BELOW
from rpy2.robjects import Formula, FactorVector
from rpy2.robjects.packages import importr

# Convert Factor variables to factors before converting dataframe to a R dataframe
# -- first create a temp dataframe in R's global space (so we can muck with it via text)
ro.globalenv['tempdf'] = data
ro.r('''
    tempdf$Subject <- factor(tempdf$Subject)
    tempdf$Condition <- factor(tempdf$Condition)
    tempdf$Difficulty <- factor(tempdf$Difficulty)
    tempdf$Session <- factor(tempdf$Session)
''')
# now create a r dataframe in python space based off of the altered tempdf
data_r = pandas2ri.DataFrame(ro.globalenv['tempdf'])

'''
^^ why not just do this in python space, e.g.:

data_r["Subject"] = FactorVector(data_r["Subject"])
data_r["Condition"] = FactorVector(data_r["Condition"])
data_r["Difficulty"] = FactorVector(data_r["Difficulty"])
data_r["Session"] = FactorVector(data_r["Session"])

that's not working....maybe someone else knows a way to achieve this
'''

# Import required R packages
# --- I need this library location and the corresponding lib_loc=library_location parameter below.
# ---- You may not need it. Even if you do, this will only be the correct location if you are 
# ---- using Linux AND version 4.4X of R.
library_location = "/home/nogard/R/x86_64-pc-linux-gnu-library/4.4"
base = importr('base')
lme4 = importr('lme4', lib_loc=library_location)
car = importr('car', lib_loc=library_location)
sjstats = importr('sjstats', lib_loc=library_location)
emmeans = importr('emmeans', lib_loc=library_location)
effects = importr('effects', lib_loc=library_location)

print(data_r.head())

# Set orthogonal contrasts (OPTIONAL)
options = ro.r('options')
op = options(contrasts = ro.StrVector(["contr.helmert", "contr.poly"]))

# Print and analyze the data
print("Mixed Effects ANOVA")
model = lme4.lmer('RT ~ Condition * Difficulty * Session + (1|Subject)', data=data_r)
anova_result = ro.r('anova')(model)
print(anova_result)

print("vvvvv This table shows p-values with type 3 sum of squares")
anova_result = car.Anova(model, type=3)  # type III sum of squares
print(anova_result)

print("\nMarginal Means")

means = emmeans.emmeans(model, specs=Formula("~ Condition * Difficulty * Session"))
print(means)

print("\nPairwise Comparisons")
comparisons = ro.r("pairs")(means)
print(comparisons)

print("\nConfidence Intervals")
print(ro.r('confint')(model))

# Reset options
options(op)


/home/nogard/GLOBAL_VENVS/stats/lib/python3.10/site-packages/rpy2/robjects/conversion.py:28: DeprecationWarning: The use of rpy2py in module rpy2.robjects.conversion is deprecated. Use rpy2.robjects.conversion.get_conversion() instead of rpy2.robjects.conversion.converter.
  warnings.warn(
R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  library ‘/usr/lib/R/site-library’ contains no packages

R[write to console]: boundary (singular) fit: see help('isSingular')



  index  Subject Condition Difficulty Session       RT
0     0 Subject1    Memory       Easy       1   20.413
1     1 Subject1    Memory       Easy       2   59.068
2     2 Subject1    Memory       Easy       3  117.023
3     3 Subject1    Memory       Easy       1 -185.325
4     4 Subject1    Memory       Easy       2  -22.216
5     5 Subject1    Memory       Easy       3 -147.410

Mixed Effects ANOVA
Analysis of Variance Table
                             npar Sum Sq Mean Sq F value
Condition                       1  28349   28349    2.88
Difficulty                      1   5869    5869    0.60
Session                         2 122543   61271    6.22
Condition:Difficulty            1  17525   17525    1.78
Condition:Session               2  11661    5830    0.59
Difficulty:Session              2 100151   50076    5.09
Condition:Difficulty:Session    2   7929    3964    0.40

vvvvv This table shows p-values with type 3 sum of squares
Analysis of Deviance Table (Type III Wald chisquare

R[write to console]: Computing profile confidence intervals ...



 contrast                                        estimate   SE  df t.ratio p.value
 Memory Easy Session1 - NoMemory Easy Session1      1.535 15.7 909   0.098  1.0000
 Memory Easy Session1 - Memory Hard Session1      -32.932 15.7 909  -2.099  0.6234
 Memory Easy Session1 - NoMemory Hard Session1    -12.080 15.7 909  -0.770  0.9998
 Memory Easy Session1 - Memory Easy Session2      -33.600 15.7 909  -2.142  0.5925
 Memory Easy Session1 - NoMemory Easy Session2    -46.389 15.7 909  -2.957  0.1232
 Memory Easy Session1 - Memory Hard Session2      -34.869 15.7 909  -2.223  0.5333
 Memory Easy Session1 - NoMemory Hard Session2    -17.733 15.7 909  -1.131  0.9933
 Memory Easy Session1 - Memory Easy Session3      -57.546 15.7 909  -3.668  0.0137
 Memory Easy Session1 - NoMemory Easy Session3    -39.323 15.7 909  -2.507  0.3368
 Memory Easy Session1 - Memory Hard Session3      -34.146 15.7 909  -2.177  0.5671
 Memory Easy Session1 - NoMemory Hard Session3    -13.893 15.7 909  -0.886  0.9992
 NoM

ValueError: Not an R object.

<rpy2.robjects.vectors.ListVector object at 0x77e0f8554b80> [RTYPES.VECSXP]
R classes: ('list',)
[StrSexpVector]
  contrasts: <class 'rpy2.rinterface_lib.sexp.StrSexpVector'>
  <rpy2.rinterface_lib.sexp.StrSexpVector object at 0x77e0f7bf9700> [RTYPES.STRSXP]